<a href="https://colab.research.google.com/github/zainikhan999/blog_writing_agents/blob/main/ReflectionAndBlogPostWriting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Blog writer**

In [1]:
!pip install autogen


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.5/562.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.4 MB/s eta 0:00:00


In [42]:
import time
import autogen


In [43]:
from google.colab import userdata
deep_seek=userdata.get('DEEP_SEEK_API')

In [44]:

# Define LLM configuration
llm_config = {
    "config_list": [
        {
            "model": "deepseek/deepseek-r1",
            "api_key": deep_seek,  # Replace with actual API key
            "base_url": "https://openrouter.ai/api/v1",
            "price": [0, 0],
        }
    ],
    "temperature": 0.7,
    "max_tokens": 1000
}



In [45]:
# Initialize the Assistant agent
writer = autogen.AssistantAgent(name="writer",
                                     system_message="You are a writer. You write engaging and concise "
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,)

In [46]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''

In [47]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [48]:
print(reply)


**Unlock the Future: Learn AI with DeepLearning.AI**  
In an era where artificial intelligence reshapes industries, DeepLearning.AI stands as a beacon for learners worldwide. Founded by Andrew Ng, a trailblazer in machine learning, this platform offers dynamic courses—from introductory deep learning to specialized tracks in NLP and healthcare AI. Hosted on Coursera, their programs combine rigorous theory with real-world projects, guided by industry experts. Whether you're a novice or a seasoned pro, DeepLearning.AI equips you with skills to innovate and lead. Join a global community pushing AI's boundaries. Start your journey today—the future awaits.  

(Word count: 99)


# Adding Reflection

In [49]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [50]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
writer (to Critic):

**Unlock AI Expertise with DeepLearning.AI**  
Founded by AI visionary Andrew Ng, DeepLearning.AI democratizes cutting-edge AI education through accessible online courses. Hosted on platforms like Coursera, its programs—from the iconic *Deep Learning Specialization* to tailored short courses—equip learners with skills in machine learning, NLP, and neural networks. Whether you’re a novice or a seasoned developer, hands-on projects and industry-relevant curricula bridge theory and practice. Join a global community reshaping healthcare, finance, and beyond with AI innovation. Ready to future-proof your skills? DeepLearning.AI turns curiosity into expertise—one algorithm at a time. Start your journey today. (99 words)

---------------

#Nested Chat

In [52]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEOReviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [59]:
legal_reviewer = autogen.AssistantAgent(
    name="LegalReviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [54]:
ethics_reviewer = autogen.AssistantAgent(
    name="EthicsReviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [56]:
meta_reviewer = autogen.AssistantAgent(
    name="MetaReviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

# Orchestrate the nested chats to solve the task

In [63]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content.
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer,
     "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer,
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
     "max_turns": 1},
]

In [64]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [65]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
writer (to Critic):

**Unlock AI Expertise with DeepLearning.AI**  
Founded by AI visionary Andrew Ng, DeepLearning.AI democratizes cutting-edge AI education through accessible online courses. Hosted on platforms like Coursera, its programs—from the iconic *Deep Learning Specialization* to tailored short courses—equip learners with skills in machine learning, NLP, and neural networks. Whether you’re a novice or a seasoned developer, hands-on projects and industry-relevant curricula bridge theory and practice. Join a global community reshaping healthcare, finance, and beyond with AI innovation. Ready to future-proof your skills? DeepLearning.AI turns curiosity into expertise—one algorithm at a time. Start your journey today. (99 words)

---------------

In [66]:
print(res.summary)

**Unlock AI Expertise with DeepLearning.AI Online Courses**  
Founded by AI pioneer Andrew Ng, DeepLearning.AI delivers beginner-friendly AI certifications and Coursera®-based courses, democratizing cutting-edge education. Programs like the *Deep Learning Specialization* and hands-on projects teach machine learning, NLP, and ethical AI practices for sectors like healthcare. Industry-aligned curricula cater to all skill levels, bridging theory and real-world impact. Ready to advance your skills?
